In [1]:
import numpy as np
import pandas as pd

import tensorflow
from tensorflow.keras import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.utils import to_categorical

In [2]:
''' defining some useful funcion '''

def loading_data(train=40000, test=10000, full=False):
    ''' reading data '''
    if full:
        s_df = pd.read_csv('sudoku.csv').values
    else:
        ''' converting s_df into array '''
        s_df = next(
            pd.read_csv('sudoku.csv', chunksize=(train + test))).values
        
    ''' transposing data '''
    q, sols = s_df.T
    X = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9)) for flatten_grid in q])
    y = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9)) for flatten_grid in sols])
    
    return (X[:train], y[:train]), (X[train:], y[train:])


def diff(grids_true, grids_pred):
    """ This function shows how well predicted quizzes fit to actual solutions. """
    return (grids_true != grids_pred).sum((1, 2))


def del_digits(X, delete=1):
    """ This function is used to create sudoku quizzes from solutions """
    grids = X.argmax(3)  # get the grid in a (9, 9) integer shape
    for grid in grids:
        grid.flat[np.random.randint(0, 81, delete)] = 0  # generate blanks (replace = True)
        
    return to_categorical(grids)


def batch_smart_solve(grids, solver):
    """ This function is ugly, feel free to optimize the code """
    grids = grids.copy()
    for _ in range((grids == 0).sum((1, 2)).max()):
        pred = np.array(solver.predict(to_categorical(grids)))  # get predictions
        probs = pred.max(2).T  # get highest probability for each 81 digit to predict
        values = pred.argmax(2).T + 1  # get corresponding values
        zeros = (grids == 0).reshape((grids.shape[0], 81))  # get blank positions

        for grid, prob, value, zero in zip(grids, probs, values, zeros):
            '''  don't try to fill already completed grid'''
            if any(zero):  
                '''focus on blanks only'''
                where = np.where(zero)[0]  
                ''' best score FOR A ZERO VALUE (confident blank) '''
                conf_pos = where[prob[zero].argmax()] 
                ''' get corresponding value '''
                conf_val = value[conf_pos]  
                ''' fill digit inplace '''
                grid.flat[conf_pos] = conf_val  
    return grids

In [3]:
inp_shape = (9, 9, 10)
(_, y_train), (X_test, y_test) = loading_data() 

''' one-hot encoding'''
X_train = to_categorical(y_train).astype('float32')
X_test = to_categorical(y_test).astype('float32')

y_train = to_categorical(y_train-1).astype('float32') 
y_test = to_categorical(y_test-1).astype('float32')   

In [4]:
''' Model '''

''' container '''
model = Sequential()

''' 1st Hidden Layer '''
model.add(Dense(64, activation='relu', input_shape=inp_shape))
model.add(Dropout(0.4))

''' 2nd Hidden Layer '''
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))

''' flattening output '''
model.add(Flatten())

''' Input layer '''
inp = Input(shape=inp_shape)  
features = model(inp)  # commons features

''' Classification Layer '''
out = [Dense(9, activation='softmax')(features) for i in range(81)]

''' final model '''
model = Model(inp, out)  

''' compile the model '''
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
''' training '''
model.fit(del_digits(X_train, 0), [y_train[:, i, j, :] for i in range(9) for j in range(9)], batch_size=128, epochs=1, 
           verbose=1)

313/313 [==============================] - 69s 133ms/step - loss: 42.7435 - dense_2_loss: 0.5219 - dense_3_loss: 0.5259 - dense_4_loss: 0.5214 - dense_5_loss: 0.5263 - dense_6_loss: 0.5284 - dense_7_loss: 0.5338 - dense_8_loss: 0.5271 - dense_9_loss: 0.5319 - dense_10_loss: 0.5251 - dense_11_loss: 0.5232 - dense_12_loss: 0.5320 - dense_13_loss: 0.5268 - dense_14_loss: 0.5237 - dense_15_loss: 0.5252 - dense_16_loss: 0.5270 - dense_17_loss: 0.5250 - dense_18_loss: 0.5308 - dense_19_loss: 0.5262 - dense_20_loss: 0.5286 - dense_21_loss: 0.5290 - dense_22_loss: 0.5239 - dense_23_loss: 0.5239 - dense_24_loss: 0.5255 - dense_25_loss: 0.5232 - dense_26_loss: 0.5282 - dense_27_loss: 0.5286 - dense_28_loss: 0.5311 - dense_29_loss: 0.5247 - dense_30_loss: 0.5279 - dense_31_loss: 0.5259 - dense_32_loss: 0.5245 - dense_33_loss: 0.5294 - dense_34_loss: 0.5254 - dense_35_loss: 0.5291 - dense_36_loss: 0.5217 - dense_37_loss: 0.5303 - dense_38_loss: 0.5284 - dense_39_loss: 0.5242 - dense_40_loss: 0.526

In [ ]:
''' Early Stopping '''
es = EarlyStopping(patience=2, verbose=1)

i = 1
for ep, dele in zip([1, 2, 3, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], [1, 2, 3, 4, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 55]):

    print('Pass n° {} ...'.format(i))
    i += 1
    
    ''' training '''
    model.fit(del_digits(X_train, dele), [y_train[:, i, j, :] for i in range(9) for j in range(9)],
              validation_data=(del_digits(X_train, dele), [y_train[:, i, j, :] for i in range(9) for j in range(9)]), 
              batch_size=128, epochs=ep, verbose=1, callbacks=[es])

Pass n° 1 ...
313/313 [==============================] - 63s 201ms/step - loss: 1.1660 - dense_2_loss: 0.0143 - dense_3_loss: 0.0143 - dense_4_loss: 0.0129 - dense_5_loss: 0.0149 - dense_6_loss: 0.0148 - dense_7_loss: 0.0141 - dense_8_loss: 0.0141 - dense_9_loss: 0.0153 - dense_10_loss: 0.0144 - dense_11_loss: 0.0157 - dense_12_loss: 0.0143 - dense_13_loss: 0.0140 - dense_14_loss: 0.0144 - dense_15_loss: 0.0158 - dense_16_loss: 0.0159 - dense_17_loss: 0.0141 - dense_18_loss: 0.0161 - dense_19_loss: 0.0154 - dense_20_loss: 0.0125 - dense_21_loss: 0.0140 - dense_22_loss: 0.0152 - dense_23_loss: 0.0127 - dense_24_loss: 0.0144 - dense_25_loss: 0.0151 - dense_26_loss: 0.0143 - dense_27_loss: 0.0151 - dense_28_loss: 0.0143 - dense_29_loss: 0.0138 - dense_30_loss: 0.0151 - dense_31_loss: 0.0137 - dense_32_loss: 0.0130 - dense_33_loss: 0.0143 - dense_34_loss: 0.0143 - dense_35_loss: 0.0146 - dense_36_loss: 0.0159 - dense_37_loss: 0.0147 - dense_38_loss: 0.0144 - dense_39_loss: 0.0138 - dense_4

Pass n° 2 ...
Epoch 1/2
313/313 [==============================] - 57s 182ms/step - loss: 1.3766 - dense_2_loss: 0.0167 - dense_3_loss: 0.0175 - dense_4_loss: 0.0183 - dense_5_loss: 0.0174 - dense_6_loss: 0.0194 - dense_7_loss: 0.0159 - dense_8_loss: 0.0172 - dense_9_loss: 0.0180 - dense_10_loss: 0.0178 - dense_11_loss: 0.0178 - dense_12_loss: 0.0172 - dense_13_loss: 0.0197 - dense_14_loss: 0.0175 - dense_15_loss: 0.0169 - dense_16_loss: 0.0172 - dense_17_loss: 0.0181 - dense_18_loss: 0.0146 - dense_19_loss: 0.0168 - dense_20_loss: 0.0178 - dense_21_loss: 0.0159 - dense_22_loss: 0.0170 - dense_23_loss: 0.0193 - dense_24_loss: 0.0149 - dense_25_loss: 0.0186 - dense_26_loss: 0.0177 - dense_27_loss: 0.0159 - dense_28_loss: 0.0175 - dense_29_loss: 0.0197 - dense_30_loss: 0.0150 - dense_31_loss: 0.0175 - dense_32_loss: 0.0156 - dense_33_loss: 0.0171 - dense_34_loss: 0.0160 - dense_35_loss: 0.0157 - dense_36_loss: 0.0164 - dense_37_loss: 0.0165 - dense_38_loss: 0.0178 - dense_39_loss: 0.0163

Epoch 2/2
313/313 [==============================] - 59s 188ms/step - loss: 1.0111 - dense_2_loss: 0.0130 - dense_3_loss: 0.0113 - dense_4_loss: 0.0127 - dense_5_loss: 0.0132 - dense_6_loss: 0.0138 - dense_7_loss: 0.0102 - dense_8_loss: 0.0120 - dense_9_loss: 0.0111 - dense_10_loss: 0.0127 - dense_11_loss: 0.0135 - dense_12_loss: 0.0127 - dense_13_loss: 0.0105 - dense_14_loss: 0.0136 - dense_15_loss: 0.0116 - dense_16_loss: 0.0124 - dense_17_loss: 0.0134 - dense_18_loss: 0.0131 - dense_19_loss: 0.0120 - dense_20_loss: 0.0133 - dense_21_loss: 0.0134 - dense_22_loss: 0.0114 - dense_23_loss: 0.0113 - dense_24_loss: 0.0138 - dense_25_loss: 0.0130 - dense_26_loss: 0.0107 - dense_27_loss: 0.0111 - dense_28_loss: 0.0130 - dense_29_loss: 0.0134 - dense_30_loss: 0.0127 - dense_31_loss: 0.0128 - dense_32_loss: 0.0120 - dense_33_loss: 0.0123 - dense_34_loss: 0.0118 - dense_35_loss: 0.0112 - dense_36_loss: 0.0109 - dense_37_loss: 0.0120 - dense_38_loss: 0.0129 - dense_39_loss: 0.0122 - dense_40_lo

Pass n° 3 ...
Epoch 1/3
313/313 [==============================] - 56s 180ms/step - loss: 1.6874 - dense_2_loss: 0.0197 - dense_3_loss: 0.0191 - dense_4_loss: 0.0196 - dense_5_loss: 0.0198 - dense_6_loss: 0.0201 - dense_7_loss: 0.0206 - dense_8_loss: 0.0222 - dense_9_loss: 0.0226 - dense_10_loss: 0.0199 - dense_11_loss: 0.0229 - dense_12_loss: 0.0218 - dense_13_loss: 0.0235 - dense_14_loss: 0.0190 - dense_15_loss: 0.0198 - dense_16_loss: 0.0196 - dense_17_loss: 0.0195 - dense_18_loss: 0.0245 - dense_19_loss: 0.0210 - dense_20_loss: 0.0205 - dense_21_loss: 0.0188 - dense_22_loss: 0.0194 - dense_23_loss: 0.0220 - dense_24_loss: 0.0216 - dense_25_loss: 0.0197 - dense_26_loss: 0.0216 - dense_27_loss: 0.0198 - dense_28_loss: 0.0194 - dense_29_loss: 0.0183 - dense_30_loss: 0.0192 - dense_31_loss: 0.0207 - dense_32_loss: 0.0202 - dense_33_loss: 0.0191 - dense_34_loss: 0.0236 - dense_35_loss: 0.0222 - dense_36_loss: 0.0208 - dense_37_loss: 0.0223 - dense_38_loss: 0.0200 - dense_39_loss: 0.0237

Epoch 2/3
313/313 [==============================] - 57s 181ms/step - loss: 1.0628 - dense_2_loss: 0.0129 - dense_3_loss: 0.0130 - dense_4_loss: 0.0133 - dense_5_loss: 0.0101 - dense_6_loss: 0.0119 - dense_7_loss: 0.0134 - dense_8_loss: 0.0146 - dense_9_loss: 0.0128 - dense_10_loss: 0.0141 - dense_11_loss: 0.0128 - dense_12_loss: 0.0118 - dense_13_loss: 0.0109 - dense_14_loss: 0.0130 - dense_15_loss: 0.0141 - dense_16_loss: 0.0145 - dense_17_loss: 0.0121 - dense_18_loss: 0.0125 - dense_19_loss: 0.0143 - dense_20_loss: 0.0142 - dense_21_loss: 0.0128 - dense_22_loss: 0.0153 - dense_23_loss: 0.0150 - dense_24_loss: 0.0129 - dense_25_loss: 0.0157 - dense_26_loss: 0.0127 - dense_27_loss: 0.0128 - dense_28_loss: 0.0133 - dense_29_loss: 0.0123 - dense_30_loss: 0.0124 - dense_31_loss: 0.0133 - dense_32_loss: 0.0120 - dense_33_loss: 0.0123 - dense_34_loss: 0.0143 - dense_35_loss: 0.0136 - dense_36_loss: 0.0140 - dense_37_loss: 0.0129 - dense_38_loss: 0.0118 - dense_39_loss: 0.0142 - dense_40_lo

...

Epoch 4: early stopping
Pass n° 9 ...
Epoch 1/10
313/313 [==============================] - 163s 521ms/step - loss: 3.1654 - dense_2_loss: 0.0378 - dense_3_loss: 0.0399 - dense_4_loss: 0.0382 - dense_5_loss: 0.0410 - dense_6_loss: 0.0391 - dense_7_loss: 0.0391 - dense_8_loss: 0.0394 - dense_9_loss: 0.0398 - dense_10_loss: 0.0399 - dense_11_loss: 0.0409 - dense_12_loss: 0.0399 - dense_13_loss: 0.0393 - dense_14_loss: 0.0391 - dense_15_loss: 0.0403 - dense_16_loss: 0.0395 - dense_17_loss: 0.0396 - dense_18_loss: 0.0406 - dense_19_loss: 0.0408 - dense_20_loss: 0.0390 - dense_21_loss: 0.0402 - dense_22_loss: 0.0385 - dense_23_loss: 0.0383 - dense_24_loss: 0.0373 - dense_25_loss: 0.0388 - dense_26_loss: 0.0388 - dense_27_loss: 0.0361 - dense_28_loss: 0.0372 - dense_29_loss: 0.0421 - dense_30_loss: 0.0388 - dense_31_loss: 0.0377 - dense_32_loss: 0.0421 - dense_33_loss: 0.0390 - dense_34_loss: 0.0395 - dense_35_loss: 0.0400 - dense_36_loss: 0.0375 - dense_37_loss: 0.0378 - dense_38_loss: 0.03

Epoch 2/10
313/313 [==============================] - 180s 577ms/step - loss: 1.9598 - dense_2_loss: 0.0241 - dense_3_loss: 0.0247 - dense_4_loss: 0.0231 - dense_5_loss: 0.0272 - dense_6_loss: 0.0260 - dense_7_loss: 0.0220 - dense_8_loss: 0.0246 - dense_9_loss: 0.0235 - dense_10_loss: 0.0234 - dense_11_loss: 0.0255 - dense_12_loss: 0.0220 - dense_13_loss: 0.0246 - dense_14_loss: 0.0249 - dense_15_loss: 0.0256 - dense_16_loss: 0.0241 - dense_17_loss: 0.0238 - dense_18_loss: 0.0235 - dense_19_loss: 0.0253 - dense_20_loss: 0.0215 - dense_21_loss: 0.0245 - dense_22_loss: 0.0281 - dense_23_loss: 0.0232 - dense_24_loss: 0.0239 - dense_25_loss: 0.0238 - dense_26_loss: 0.0239 - dense_27_loss: 0.0227 - dense_28_loss: 0.0236 - dense_29_loss: 0.0265 - dense_30_loss: 0.0242 - dense_31_loss: 0.0250 - dense_32_loss: 0.0269 - dense_33_loss: 0.0221 - dense_34_loss: 0.0243 - dense_35_loss: 0.0234 - dense_36_loss: 0.0252 - dense_37_loss: 0.0221 - dense_38_loss: 0.0226 - dense_39_loss: 0.0237 - dense_40_

Epoch 3/10
313/313 [==============================] - 160s 511ms/step - loss: 1.6381 - dense_2_loss: 0.0204 - dense_3_loss: 0.0212 - dense_4_loss: 0.0205 - dense_5_loss: 0.0207 - dense_6_loss: 0.0211 - dense_7_loss: 0.0210 - dense_8_loss: 0.0223 - dense_9_loss: 0.0196 - dense_10_loss: 0.0202 - dense_11_loss: 0.0197 - dense_12_loss: 0.0205 - dense_13_loss: 0.0191 - dense_14_loss: 0.0206 - dense_15_loss: 0.0205 - dense_16_loss: 0.0196 - dense_17_loss: 0.0194 - dense_18_loss: 0.0211 - dense_19_loss: 0.0199 - dense_20_loss: 0.0210 - dense_21_loss: 0.0209 - dense_22_loss: 0.0219 - dense_23_loss: 0.0205 - dense_24_loss: 0.0201 - dense_25_loss: 0.0209 - dense_26_loss: 0.0199 - dense_27_loss: 0.0207 - dense_28_loss: 0.0197 - dense_29_loss: 0.0220 - dense_30_loss: 0.0198 - dense_31_loss: 0.0198 - dense_32_loss: 0.0212 - dense_33_loss: 0.0200 - dense_34_loss: 0.0199 - dense_35_loss: 0.0205 - dense_36_loss: 0.0213 - dense_37_loss: 0.0196 - dense_38_loss: 0.0207 - dense_39_loss: 0.0198 - dense_40_

Epoch 4/10
313/313 [==============================] - 151s 483ms/step - loss: 1.4696 - dense_2_loss: 0.0170 - dense_3_loss: 0.0175 - dense_4_loss: 0.0183 - dense_5_loss: 0.0186 - dense_6_loss: 0.0216 - dense_7_loss: 0.0188 - dense_8_loss: 0.0177 - dense_9_loss: 0.0163 - dense_10_loss: 0.0183 - dense_11_loss: 0.0190 - dense_12_loss: 0.0186 - dense_13_loss: 0.0198 - dense_14_loss: 0.0182 - dense_15_loss: 0.0177 - dense_16_loss: 0.0178 - dense_17_loss: 0.0173 - dense_18_loss: 0.0178 - dense_19_loss: 0.0182 - dense_20_loss: 0.0195 - dense_21_loss: 0.0195 - dense_22_loss: 0.0185 - dense_23_loss: 0.0178 - dense_24_loss: 0.0189 - dense_25_loss: 0.0191 - dense_26_loss: 0.0158 - dense_27_loss: 0.0195 - dense_28_loss: 0.0178 - dense_29_loss: 0.0188 - dense_30_loss: 0.0170 - dense_31_loss: 0.0188 - dense_32_loss: 0.0181 - dense_33_loss: 0.0171 - dense_34_loss: 0.0177 - dense_35_loss: 0.0179 - dense_36_loss: 0.0178 - dense_37_loss: 0.0181 - dense_38_loss: 0.0181 - dense_39_loss: 0.0163 - dense_40_

Epoch 4: early stopping
Pass n° 10 ...
Epoch 1/10
313/313 [==============================] - ETA: 0s - loss: 5.1848 - dense_2_loss: 0.0627 - dense_3_loss: 0.0639 - dense_4_loss: 0.0649 - dense_5_loss: 0.0657 - dense_6_loss: 0.0650 - dense_7_loss: 0.0644 - dense_8_loss: 0.0656 - dense_9_loss: 0.0628 - dense_10_loss: 0.0628 - dense_11_loss: 0.0654 - dense_12_loss: 0.0609 - dense_13_loss: 0.0654 - dense_14_loss: 0.0613 - dense_15_loss: 0.0639 - dense_16_loss: 0.0650 - dense_17_loss: 0.0635 - dense_18_loss: 0.0657 - dense_19_loss: 0.0624 - dense_20_loss: 0.0637 - dense_21_loss: 0.0658 - dense_22_loss: 0.0620 - dense_23_loss: 0.0640 - dense_24_loss: 0.0639 - dense_25_loss: 0.0604 - dense_26_loss: 0.0631 - dense_27_loss: 0.0647 - dense_28_loss: 0.0668 - dense_29_loss: 0.0608 - dense_30_loss: 0.0624 - dense_31_loss: 0.0654 - dense_32_loss: 0.0636 - dense_33_loss: 0.0640 - dense_34_loss: 0.0608 - dense_35_loss: 0.0628 - dense_36_loss: 0.0667 - dense_37_loss: 0.0643 - dense_38_loss: 0.0622 - de

In [ ]:
'''Evaluating model '''
q = X_test.argmax(3)  
actual = y_test.argmax(3) + 1 

''' make some guesses'''
sg = batch_smart_solve(q, model)   

''' get number of errors on each quizz '''
de = diff(actual, sg)  

'''portion of correct solved quizzes '''
acc = (de == 0).mean()  

In [ ]:
print("""Grid solved:\t {} Correct ones:\t {} Accuracy:\t {}""".format(de.shape[0], (de==0).sum(), acc))